# Background
# Online Face detection and Recognition system
## Requirements: 
### 1. Able to create new user accounts with GUI
### 2. Detect and recognize a person with a live camera

# Workflow: User Registration, Model Training, and Login

This workflow involves three main steps: 
1. **User Registration:** Capture images of the user's face to create a dataset.
2. **Model Training:** Train a machine learning model (Support Vector Machine) on the collected face images.
3. **Login:** Authenticate users based on their facial features.

## Step 1: Setup Environment

In [1]:
!pip install numpy opencv-python scikit-learn pandas matplotlib

In [11]:
!pip install opencv-python-headless scikit-learn numpy

## Step 2: Import Libraries

In [34]:
import tkinter as tk
from tkinter import simpledialog, messagebox
import cv2
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import time

## Step 3: GUI for User Interaction


In [79]:
# Setting up file paths for user images, users CSV, model, PCA, and label dictionary
user_images_dir = 'user_images'
users_csv = 'users.csv'
model_filename = 'svm_face_model.pkl'
pca_filename = 'pca.pkl'
label_dict_filename = 'label_dict.csv'

# Creating user images directory if it doesn't exist
if not os.path.exists(user_images_dir):
    os.makedirs(user_images_dir)

# Ensuring the users CSV exists, if not, creating it with headers
if not os.path.exists(users_csv):
    with open(users_csv, 'w') as file:
        file.write('username,path\n')

# Function to capture images for a new user
def capture_images(username):
    # Initializing the webcam and face cascade classifier
    cap = cv2.VideoCapture(0)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    count = 0  # Counter for captured images
    
    # Creating directory for the current user's images
    user_dir = os.path.join(user_images_dir, username)
    if not os.path.exists(user_dir):
        os.makedirs(user_dir)

    while True:
        ret, frame = cap.read()  # Reading frame from webcam
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Converting frame to grayscale
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)  # Detecting faces in the frame

        for (x, y, w, h) in faces:
            # Drawing rectangle around detected face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            face = gray[y:y+h, x:x+w]  # Extracting face region
            face = cv2.resize(face, (100, 100))  # Resizing face image
            if count < 100:  # Take 100 samples
                cv2.imwrite(os.path.join(user_dir, f'{count}.png'), face)  # Saving face image
                count += 1

        cv2.imshow('Capture Images - Press q to quit', frame)  # Displaying frame with rectangle around face
        if cv2.waitKey(1) & 0xFF == ord('q') or count >= 100:  # Exiting loop if 'q' is pressed or 100 samples taken
            break
    
    cap.release()  # Releasing webcam
    cv2.destroyAllWindows()  # Closing OpenCV windows

    # Updating users CSV with the new user's information
    with open(users_csv, 'a') as file:
        file.write(f'{username},{user_dir}\n')

# Function to train the model
def train_model():
    X, y, label_dict = load_images_and_labels()  # Loading images and labels
    pca = PCA(n_components=min(100, len(X))).fit(X)  # Fitting PCA with minimum of 100 components or total available images
    X_pca = pca.transform(X)  # Transforming images using PCA
    
    X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.25, random_state=42)  # Splitting dataset
    
    clf = SVC(kernel='linear', probability=True)  # Initializing SVM classifier
    clf.fit(X_train, y_train)  # Training the model
    
    # Saving the trained model, PCA, and label dictionary to files
    joblib.dump(clf, model_filename)
    joblib.dump(pca, pca_filename)
    pd.DataFrame.from_dict(label_dict, orient='index').to_csv(label_dict_filename, header=False)

    # Evaluating the model and printing classification report and confusion matrix
    y_pred = clf.predict(X_test)
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

# Function to load images and labels
def load_images_and_labels():
    images = []  # List to store images
    labels = []  # List to store labels
    label_dict = {}  # Dictionary to store label-user mapping
    users_df = pd.read_csv(users_csv)  # Reading users CSV
    
    for index, row in users_df.iterrows():
        user_folder = row['path']  # Path to user's image directory
        username = row['username']  # User's name
        label = index  # Label for the user
        label_dict[label] = username  # Mapping label to username
        
        for image_file in os.listdir(user_folder):
            image_path = os.path.join(user_folder, image_file)  # Path to the image file
            image = cv2.imread(image_path, 0)  # Reading image in grayscale
            images.append(image)  # Appending image to list
            labels.append(label)  # Appending label to list
    
    X = np.array(images).reshape(len(images), -1)  # Converting image list to numpy array
    y = np.array(labels)  # Converting label list to numpy array
    return X, y, label_dict  # Returning images, labels, and label dictionary

# Function to login using Face ID
def login_user():
    username = simpledialog.askstring("Login", "Enter username to login:")  # Prompting user for username
    if not username:
        messagebox.showinfo("Login", "No username entered.")  # Showing info message if no username entered
        return

    clf = joblib.load(model_filename)  # Loading trained model
    pca = joblib.load(pca_filename)  # Loading PCA
    label_dict = pd.read_csv(label_dict_filename, header=None, index_col=0).squeeze("columns").to_dict()  # Loading label dictionary
    
    target_label = None  # Initializing target label
    for label, user in label_dict.items():
        if user == username:
            target_label = label  # Getting label for the given username
            break
    
    if target_label is None:
        messagebox.showinfo("Login", f"Username '{username}' not found in trained model.")  # Showing info message if username not found
        return

    cap = cv2.VideoCapture(0)  # Initializing webcam
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')  # Initializing face cascade classifier
    detected = False  # Initializing detection flag
    start_time = time.time()  # Start time for timer

    
    while True:
        ret, frame = cap.read()  # Reading frame from webcam
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Converting frame to grayscale
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)  # Detecting faces in the frame

        for (x, y, w, h) in faces:
            face = gray[y:y+h, x:x+w]  # Extracting face region
            face = cv2.resize(face, (100, 100)).reshape(1, -1)  # Resizing and flattening face image
            face_pca = pca.transform(face)  # Transforming face image using PCA
            pred = clf.predict(face_pca)  # Predicting label for the face
            prob = clf.predict_proba(face_pca).max()  # Getting maximum probability
            
            if pred[0] == target_label and prob > 0.90:  # Checking if predicted label matches target label and probability is high enough
                cv2.putText(frame, f'{username} {prob*100:.2f}%', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)  # Displaying username and probability
                detected = True  # Setting detection flag to True
            else:
                cv2.putText(frame, f'{username} not detected', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)  # Displaying username not detected
                if time.time() - start_time >= 5:  # Exiting loop if username not detected for 5 seconds
                    messagebox.showinfo("Login", f"Username '{username}' not detected. Login aborted.")
                    cap.release()  # Releasing webcam
                    cv2.destroyAllWindows()  # Closing OpenCV windows
                    return
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)  # Drawing rectangle around face
        
        cv2.imshow('Login - Press q to quit', frame)  # Displaying frame
        key = cv2.waitKey(1)
        
        if detected:
            cv2.waitKey(2000)  # Waiting for 2 seconds
            messagebox.showinfo("Login Successful", f"{username} logged in.")  # Showing info message for successful login
            break

        if key & 0xFF == ord('q'):  # Exiting loop if 'q' is pressed
            break
    
    cap.release()  # Releasing webcam
    cv2.destroyAllWindows()  # Closing OpenCV windows


## Step 4: Start GUI

In [80]:
# Final GUI function
def app_gui():
    root = tk.Tk()
    root.title("Face Detection and Recognition System")

    create_user_btn = tk.Button(root, text="Create New User", command=create_user)
    create_user_btn.pack(pady=20)

    login_btn = tk.Button(root, text="Face ID Login", command=login_user)
    login_btn.pack(pady=20)

    train_btn = tk.Button(root, text="Train Model", command=train_model)
    train_btn.pack(pady=20)

    root.mainloop()

app_gui()

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        27
           1       1.00      1.00      1.00        23

    accuracy                           1.00        50
   macro avg       1.00      1.00      1.00        50
weighted avg       1.00      1.00      1.00        50

[[27  0]
 [ 0 23]]
